In [14]:
! make -C mpy-cross clean
! make -C mpy-cross
# ! make -C ports/unix clean
# ! make -C ports/unix CFLAGS_EXTRA="-DMICROPY_PY_SYS_SETTRACE=1"

make: Entering directory '/home/jos/micropython/mpy-cross'
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
rm -f build/mpy-cross
rm -f build/mpy-cross.map
rm -rf build 
make: Leaving directory '/home/jos/micropython/mpy-cross'
make: Entering directory '/home/jos/micropython/mpy-cross'
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
mkdir -p build/genhdr
GEN build/genhdr/mpversion.h
GEN build/genhdr/qstr.i.last
GEN build/genhdr/qstr.split
GEN build/genhdr/qstrdefs.collected.h
QSTR updated
GEN build/genhdr/qstrdefs.generated.h
GEN build/genhdr/moduledefs.split
GEN build/genhdr/moduledefs.collected
Module registrations updated
GEN build/genhdr/moduledefs.h
GEN build/genhdr/root_pointers.split
GEN build/genhdr/root_pointers.collected
Root pointer registrations updated
GEN build/genhdr/root_pointers.h
mkdir -p build/py/
mkdir -p build/shared/runtime/
CC ../py/mpstate.c
CC ../py/nlr.c
CC ../py/nlrx86.c
CC ../py/nlrx64.c
CC

```c
// Enable sys.settrace support
#define MICROPY_PY_SYS_SETTRACE            (1)

// Disable compiler optimizations for debugging
#define MICROPY_COMP_CONST                 (0)
#define MICROPY_COMP_MODULE_CONST          (0)
#define MICROPY_COMP_DOUBLE_TUPLE_ASSIGN   (0)
#define MICROPY_COMP_TRIPLE_TUPLE_ASSIGN   (0)
```

In [15]:
! make -C ports/unix clean


make: Entering directory '/home/jos/micropython/ports/unix'
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
rm -f build-standard/micropython
rm -f build-standard/micropython.map
rm -rf build-standard 
make: Leaving directory '/home/jos/micropython/ports/unix'


In [16]:
# with modified mpportconfig
! make -C ports/unix 

make: Entering directory '/home/jos/micropython/ports/unix'
Use make V=1 or set BUILD_VERBOSE in your environment to increase build verbosity.
mkdir -p build-standard/genhdr
GEN build-standard/genhdr/mpversion.h
GEN build-standard/genhdr/qstr.i.last
GEN build-standard/genhdr/qstr.split
GEN build-standard/genhdr/qstrdefs.collected.h
QSTR updated
GEN build-standard/genhdr/qstrdefs.generated.h
GEN build-standard/genhdr/moduledefs.split
GEN build-standard/genhdr/moduledefs.collected
Module registrations updated
GEN build-standard/genhdr/moduledefs.h
GEN build-standard/genhdr/root_pointers.split
GEN build-standard/genhdr/root_pointers.collected
Root pointer registrations updated
GEN build-standard/genhdr/root_pointers.h
GEN build-standard/genhdr/compressed.split
GEN build-standard/genhdr/compressed.collected
Compressed data updated
GEN build-standard/genhdr/compressed.data.h
mkdir -p build-standard/extmod/
mkdir -p build-standard/extmod/mbedtls/
mkdir -p build-standard/lib/berkeley-db-1.xx/

In [17]:
%%writefile test_settrace.py
import sys
import json
from collections import OrderedDict as OD

def tracer(frame, event, arg):
    try:
        # print(repr(frame)   )
        # print(dir(frame))
        # for name in dir(frame):
        #         print(f"{name} = {getattr(frame, name)}")
        # # print(f"{json.dumps(frame)}")
        # print(f"{frame.f_globals=}")
        print(f"( {frame.f_lineno} , {frame.f_lasti} ) ", end="")
        # sort frame.f_locals by key , and add all k,v pairs to an OrderedDict
        f_locals = OD(sorted(frame.f_locals.items()))
        print(f"{f_locals=}")
        print( type(f_locals[list(f_locals)[0]]))

    except Exception as e:
        pass

    return tracer

granny = "granny"

def test():
    foo = "hello debugger"
    _under = "this is an under variable"
    __dunder = "this is a double under variable"
    bar = granny
    a = 100
    b = 2200
    c = 33333
    l = [1, 2, 3]
    d = {"a": 1, "b": 2, "c": 3}
    tuple1 = (1, 2, 3)
    tuple2 = (a, b, c,a)
    print(foo)
    print(a+b+c)
    long_name = a+ b + c
    print(long_name)

sys.settrace(tracer)
test()
sys.settrace(None)

Overwriting test_settrace.py


In [18]:
! ./ports/unix/build-standard/micropython  test_settrace.py

( 26 , 0 ) f_locals=OrderedDict({})
( 27 , 0 ) f_locals=OrderedDict({})
( 28 , 0 ) f_locals=OrderedDict({'foo': 'hello debugger', 'local_16': 'hello debugger'})
<class 'str'>
( 29 , 0 ) f_locals=OrderedDict({'foo': 'this is an under variable', 'local_15': 'this is an under variable', 'local_16': 'hello debugger'})
<class 'str'>
( 30 , 0 ) f_locals=OrderedDict({'foo': 'this is a double under variable', 'local_14': 'this is a double under variable', 'local_15': 'this is an under variable', 'local_16': 'hello debugger'})
<class 'str'>
( 31 , 0 ) f_locals=OrderedDict({'foo': 'granny', 'local_13': 'granny', 'local_14': 'this is a double under variable', 'local_15': 'this is an under variable', 'local_16': 'hello debugger'})
<class 'str'>
( 32 , 0 ) f_locals=OrderedDict({'foo': 100, 'local_13': 'granny', 'local_14': 'this is a double under variable', 'local_15': 'this is an under variable', 'local_16': 'hello debugger', 'long_name': 100})
<class 'int'>
( 33 , 0 ) f_locals=OrderedDict({'foo':